In [47]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

ProgrammingError: Oracle Client library has already been initialized

In [48]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [49]:
%%time

query2 = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    ,O.STATE
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,STATE
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID
                    ,STATE


"""

query = """

select distinct DTN, WIRE_CENTER_ID
from LQSMO.LU_CIRCUITS_O

"""


df_ora = pd.read_sql(query, con=conn) #look up for wc to dtn
df_ora2 = pd.read_sql(query2, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 375 ms
Wall time: 1min


In [50]:
%%time

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in (
                                          '1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494','1189253', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306',-- MPLSMNBE 
                                          '1180028', '1178694', '1179569','1180333',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924','1179922', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237','1180468', -- CRYSMNCR
                                          '1180315','1189074','1189058', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507','1180076','1180079','1180492','1180497','1180486',  --TCSNAZMA
                                          '1180091','1179913','1189283',  --TCSNAZMA
                                          '1179641','1179914', --TCSNAZCR
                                          '1179840','1180355', --STTLWA04
                                          '1180086','1180111','1180327','1180436', --STTLWASU
                                          '1179576','1179672','1179971','1179977','1179979','1179981','1179987', --CLSPCOMA
                                          '1179988','1179989','1179994','1180005','1180006','1180041','1180123', --CLSPCOMA
                                          '1189130', --MLWKOR17
                                          '1179352','1180002','1189088', --MAGNUTNM
                                          '1179858','1180015','1180098','1180392','1180399', --SPKNWA01
                                          '1180027', --BRCTMNBC
                                          '1180491','1180509', --PHNXAZNO
                                          '1180488','1180496', --TCSNAZRN
                                          '1189200' --DNVRCOCL     
                                            ) --add in MG polygon IDs here
                    
            )
            
            SELECT  WIRE_CENTER_ID
                    ,count(distinct LUID) as cnt
                    
            FROM    QD
            
            GROUP BY
                    WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 15.6 ms
Wall time: 495 ms


In [51]:
mg_lu_ora

,WIRE_CENTER_ID,CNT
0,CRYSMNCR,1275
1,MPLSMNBE,4690
2,MPLSMNGE,1772
3,PTLDOR11,264
4,STTLWA04,298
5,BRCTMNBC,363
6,STTLWASU,492
7,PHNXAZNO,243
8,MPLSMNTF,929
9,PHNXAZSO,1624


In [52]:
mg_lu_df = mg_lu_ora.rename(columns = {'CNT': 'MG_ENABLED_LU_CNT'})

In [53]:
# Input excel MG raw data
sf_df = pd.read_excel(r'Salesforce Data/salesdata.xlsx',sheet_name = 'Ella XGS Sales') 

C:\Users\AD21236\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [54]:
sf_df['Purchased Data Rate'].fillna('missing_speed', inplace=True)

In [55]:
# before import salesforce DTN field needs to be string
sf_df['DTN'] = sf_df['DTN'].apply(lambda x: '{:.0f}'.format(x))
sf_df['DTN'] = sf_df['DTN'].astype(str)

In [56]:
# select only account status Active or Pending Activation
sf_df = sf_df[(sf_df['Account Status'] == 'Active')|(sf_df['Account Status'] == 'Pending Activation')]

In [57]:
df_ora2.head()

,WIRE_CENTER_ID,STATE,CNT
0,MPLSMNTF,MN,929
1,HLDYUTMA,UT,265
2,NFMYFLXB,FL,1974
3,ISNTMNIS,MN,65
4,PHNXAZSO,AZ,1624


In [58]:
# this has state/wirecenter/Enabled LU counts at wire center level
df_wc_state = df_ora2.copy()

In [59]:
#Adding wire center detail to Salesforce
combine_df = df_ora.merge(sf_df, how = 'outer', on = 'DTN') #combine now has wire center id
combine_df.head()

,DTN,WIRE_CENTER_ID,Wire Center,Completion Date,Account Name,Service Address,Service State Short,Polygon ID,Purchased Data Rate,Created Date,Account Status,Order Number,Work Order Number,Status,Appointment Number,Due Date,Created By: Full Name
0,1000195204,DVNPIAEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000183525,BRCTMNBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000285986,BRCTMNBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000186209,BRCTMNBC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1500074404,TCSNAZFW,TCSNAZFW,12/7/2022 3:52 PM,Emmanuel PANIAGUA,832 W Edgewater Dr Tucson AZ US 85704,AZ,1180032.0,940M/940M,12/2/2022,Active,A00207130,932592.0,Completed,SA-2902237,12/7/2022 12:40 PM,SFCC Integration


In [60]:
#Adding LU count detail to Sales force combine_df
combine_df_details = df_wc_state.merge(combine_df, how = 'outer', on = 'WIRE_CENTER_ID')
combine_df_details['STATE'].value_counts() #check for nulls

FL    20643
NV     3057
CO     3057
MN     2869
AZ     2651
UT     1590
WA      624
OR       11
Name: STATE, dtype: int64

In [61]:
#Update CNT column name
combine_df_details = combine_df_details.rename(columns = {'CNT': 'Enabled_LUs_WireCenter_Level'})

In [62]:
summary_df = combine_df_details.copy()
summary_df = summary_df.fillna(0)

In [63]:
summary_df['Order Number'] = np.where(summary_df['Order Number'] == 0.0, np.NaN, summary_df['Order Number'])

In [64]:
summary_df.head()

,WIRE_CENTER_ID,STATE,Enabled_LUs_WireCenter_Level,DTN,Wire Center,Completion Date,Account Name,Service Address,Service State Short,Polygon ID,Purchased Data Rate,Created Date,Account Status,Order Number,Work Order Number,Status,Appointment Number,Due Date,Created By: Full Name
0,MPLSMNTF,MN,929.0,1500049986,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
1,MPLSMNTF,MN,929.0,1500058529,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
2,MPLSMNTF,MN,929.0,1500073847,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
3,MPLSMNTF,MN,929.0,1500042767,0,0,0,0,0,0.0,0,0,0,NaN,0.0,0,0,0,0
4,MPLSMNTF,MN,929.0,1500053112,MPLSMNTF,10/3/2022 2:33 PM,Mary Buscher,2844 43rd Ave S Minneapolis MN United States 5...,MN,1179688.0,200M/200M,9/25/2022,Active,A00180562,731884.0,Completed,SA-1620397,10/3/2022 2:20 PM,SFCC Integration


In [65]:
# check if new speeds are popping up
summary_df['Purchased Data Rate'].value_counts()

0            140571
940M/940M       979
200M/200M       515
1G/1G           261
3G/3G             9
Name: Purchased Data Rate, dtype: int64

In [66]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# summary_df['Due Date'].value_counts()

In [67]:
enablement_base_group = summary_df.groupby(['STATE','WIRE_CENTER_ID'], as_index=False)

In [68]:
# create base enablement for wire centers
enablement_base_state = enablement_base_group['Enabled_LUs_WireCenter_Level'].max()

In [69]:
from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)
# wire center level - if new speeds pop up, add to below list to create new columns
#combine2_sum_df = combine2_df.groupby(by = ['WIRE_CENTER_ID', 'Purchased Data Rate', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Work Order Number'].nunique()

# state level - get total purchases
# combine2_state_df.groupby(by = ['STATE_CD', 'Purchased Data Rate'], as_index = False)['Work Order Number'].nunique()
speed_940m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m = speed_940m.rename(columns = {'Order Number': '940M - Active Subscribers'})

# speed_940m_pending = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_940m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '940M/940M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_940m_pending = speed_940m_pending.rename(columns = {'Order Number': '940M - Pending Activation Subscribers'})

# 200M

speed_200m = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_200m = speed_200m.rename(columns = {'Order Number': '200M - Active Subscribers'})

speed_200m_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                                &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                                &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '200M/200M'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_200m_pending = speed_200m_pending.rename(columns = {'Order Number': '200M - Pending Activation Subscribers'})

# 1G

speed_1g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_1g = speed_1g.rename(columns = {'Order Number': '1G - Active Subscribers'})

speed_1g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '1G/1G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_1g_pending = speed_1g_pending.rename(columns = {'Order Number': '1G - Pending Activation Subscribers'})

# 3G
speed_3g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_3g = speed_3g.rename(columns = {'Order Number': '3G - Active Subscribers'})

speed_3g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '3G/3G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_3g_pending = speed_3g_pending.rename(columns = {'Order Number': '3G - Pending Activation Subscribers'})

# 8G

speed_8g = summary_df[((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Active'))&((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()
speed_8g = speed_8g.rename(columns = {'Order Number': '8G - Active Subscribers'})

speed_8g_pending = summary_df[((summary_df['Due Date'] == 0)| \
                                 ((pd.to_datetime(summary_df['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(summary_df['Due Date']).dt.date <= future_date))) \
                              &((summary_df['Account Status'] == 0)|(summary_df['Account Status'] == 'Pending Activation')) \
                              &((summary_df['Purchased Data Rate'] == 0)|(summary_df['Purchased Data Rate'] == '8G/8G'))].groupby(by = ['STATE', 'WIRE_CENTER_ID', 'Enabled_LUs_WireCenter_Level'], as_index = False)['Order Number'].nunique()

speed_8g_pending = speed_8g_pending.rename(columns = {'Order Number': '8G - Pending Activation Subscribers'})

In [70]:
final_summary = enablement_base_state.merge(speed_8g_pending.merge(speed_8g.merge(speed_3g_pending.merge((speed_3g.merge((speed_1g_pending.merge((speed_1g.merge((speed_200m_pending.merge((speed_200m.merge((speed_940m.merge(speed_940m_pending, how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level']), how = 'outer', on = ['STATE','WIRE_CENTER_ID','Enabled_LUs_WireCenter_Level'])


In [71]:
final_summary = final_summary.fillna(0)

In [72]:
# final_summary['Aggregate 200M Penetration Rate'] = final_summary['200M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 940M Penetration Rate'] = final_summary['940M - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 1G Penetration Rate'] = final_summary['1G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']
# final_summary['Aggregate 3G Penetration Rate'] = final_summary['3G - Active Subscribers']/final_summary['Enabled_LUs_WireCenter_Level']

In [73]:
final_summary['Total Active Subscribers'] = final_summary['8G - Active Subscribers'] + final_summary['3G - Active Subscribers'] + final_summary['1G - Active Subscribers'] + final_summary['200M - Active Subscribers'] + final_summary['940M - Active Subscribers']

In [74]:
final_summary['Total Pending Activation Subscribers'] = final_summary['8G - Pending Activation Subscribers'] + final_summary['3G - Pending Activation Subscribers'] + final_summary['1G - Pending Activation Subscribers'] + final_summary['200M - Pending Activation Subscribers'] + final_summary['940M - Pending Activation Subscribers']

In [75]:
final_summary['Aggregate Penetration Rate (Active Subs)'] = final_summary['Total Active Subscribers'] / final_summary['Enabled_LUs_WireCenter_Level']

In [76]:
# uncomment to check data
# final_summary[final_summary['Total Pending Activation Subscribers']>0]

In [77]:
# join wire center level mg enabled lu count

final_summary_mg = final_summary.merge(mg_lu_df, on = 'WIRE_CENTER_ID', how = 'left')
final_summary_mg['MG_ENABLED_LU_CNT'] = final_summary_mg['MG_ENABLED_LU_CNT'].fillna(0)
final_summary_mg = final_summary_mg.rename(columns = {'MG_ENABLED_LU_CNT': 'Multigig_Enabled_LUs'})

In [78]:
state_summary = final_summary_mg.groupby(['STATE'], as_index = False).sum()

In [79]:
state_summary = state_summary[['STATE','Enabled_LUs_WireCenter_Level','Multigig_Enabled_LUs','8G - Pending Activation Subscribers','8G - Active Subscribers','3G - Pending Activation Subscribers','3G - Active Subscribers','1G - Pending Activation Subscribers','1G - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers','200M - Pending Activation Subscribers','200M - Active Subscribers','Total Active Subscribers','Total Pending Activation Subscribers']]

In [80]:
# add state_summary aggregate penetration rate

state_summary['Aggregate Penetration Rate (Active Subs)'] = state_summary['Total Active Subscribers']/state_summary['Enabled_LUs_WireCenter_Level']

In [81]:
state_summary

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,8G - Pending Activation Subscribers,8G - Active Subscribers,3G - Pending Activation Subscribers,3G - Active Subscribers,1G - Pending Activation Subscribers,1G - Active Subscribers,940M - Active Subscribers,940M - Pending Activation Subscribers,200M - Pending Activation Subscribers,200M - Active Subscribers,Total Active Subscribers,Total Pending Activation Subscribers,Aggregate Penetration Rate (Active Subs)
0,0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,33.0,26.0,0.0,0.0,70.0,NaN
1,AZ,16987.0,14316.0,0.0,0.0,0.0,1.0,7.0,69.0,127.0,17.0,14.0,90.0,287.0,38.0,0.016895
2,CO,9276.0,4920.0,0.0,0.0,1.0,2.0,2.0,34.0,72.0,12.0,14.0,54.0,162.0,29.0,0.017464
3,FL,27539.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,155.0,275.0,142.0,45.0,200.0,417.0,0.007262
4,MN,10508.0,9711.0,0.0,0.0,0.0,2.0,30.0,62.0,41.0,4.0,12.0,54.0,159.0,46.0,0.015131
5,NV,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,1.0,1.0,8.0,0.010638
6,OR,349.0,349.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.002865
7,UT,4060.0,659.0,0.0,0.0,0.0,0.0,1.0,1.0,43.0,17.0,7.0,18.0,62.0,25.0,0.015271
8,WA,2450.0,2080.0,0.0,0.0,0.0,2.0,11.0,27.0,83.0,19.0,1.0,3.0,115.0,31.0,0.046939


In [82]:
# create MG state summary
# 10/12/22 - comment below out to allow null states to be counted
state_summary_mg = state_summary[(state_summary['Multigig_Enabled_LUs']>0) | (state_summary['STATE'] == 0)]

In [83]:
state_summary_mg = state_summary_mg.drop(['Aggregate Penetration Rate (Active Subs)','Enabled_LUs_WireCenter_Level','200M - Pending Activation Subscribers','200M - Active Subscribers','940M - Active Subscribers','940M - Pending Activation Subscribers'], axis = 1)

In [84]:
state_summary_mg['Total Active Subscribers'] = state_summary_mg['8G - Active Subscribers'] + state_summary_mg['3G - Active Subscribers'] + state_summary_mg['1G - Active Subscribers']

In [85]:
state_summary_mg['Total Pending Activation Subscribers'] = state_summary_mg['8G - Pending Activation Subscribers'] + state_summary_mg['3G - Pending Activation Subscribers'] + state_summary_mg['1G - Pending Activation Subscribers'] 

In [86]:
state_summary_mg['MG Penetration Rate (Active Subs)'] = state_summary_mg['Total Active Subscribers'] / state_summary_mg['Multigig_Enabled_LUs']

In [87]:
# reorder columns
columns1 = ['STATE', 'Multigig_Enabled_LUs', 'Total Active Subscribers',
       'Total Pending Activation Subscribers',
       'MG Penetration Rate (Active Subs)',
       '8G - Active Subscribers', '8G - Pending Activation Subscribers',
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers']

state_summary_mg = state_summary_mg[columns1]

columns2 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
        'Total Active Subscribers', 'Total Pending Activation Subscribers',
       'Aggregate Penetration Rate (Active Subs)',
       '8G - Active Subscribers','8G - Pending Activation Subscribers', 
       '3G - Active Subscribers','3G - Pending Activation Subscribers', 
       '1G - Active Subscribers','1G - Pending Activation Subscribers', 
       '940M - Active Subscribers', '940M - Pending Activation Subscribers',
        '200M - Active Subscribers','200M - Pending Activation Subscribers']

state_summary = state_summary[columns2]

columns3 = ['STATE', 'Enabled_LUs_WireCenter_Level', 'Multigig_Enabled_LUs',
            '200M - Active Subscribers',
            '940M - Active Subscribers', 
            '1G - Active Subscribers',
            '3G - Active Subscribers',
            '8G - Active Subscribers',
            'Total Active Subscribers'
]

state_summary_view = state_summary[columns3]

In [88]:
# ready for export
state_summary_view = state_summary_view[state_summary_view['STATE'] != 0].sort_values(by = 'Enabled_LUs_WireCenter_Level', ascending = False)

In [89]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Salesforce_Summary_StateWCLevel.xlsx"

with pd.ExcelWriter(excelfilename) as writer:  
    state_summary.to_excel(writer, sheet_name='State_Summary', index = False)
    state_summary_mg.to_excel(writer, sheet_name='MultiGig_State_Summary', index = False)
    final_summary.to_excel(writer, sheet_name='Summary', index = False)
    sf_df.to_excel(writer, sheet_name='Details', index = False)




### Fill template

In [90]:
# check number of rows = 8 otherwise move grand total position in templates:
state_summary_view

,STATE,Enabled_LUs_WireCenter_Level,Multigig_Enabled_LUs,200M - Active Subscribers,940M - Active Subscribers,1G - Active Subscribers,3G - Active Subscribers,8G - Active Subscribers,Total Active Subscribers
3,FL,27539.0,0.0,45.0,155.0,0.0,0.0,0.0,200.0
1,AZ,16987.0,14316.0,90.0,127.0,69.0,1.0,0.0,287.0
4,MN,10508.0,9711.0,54.0,41.0,62.0,2.0,0.0,159.0
2,CO,9276.0,4920.0,54.0,72.0,34.0,2.0,0.0,162.0
7,UT,4060.0,659.0,18.0,43.0,1.0,0.0,0.0,62.0
8,WA,2450.0,2080.0,3.0,83.0,27.0,2.0,0.0,115.0
6,OR,349.0,349.0,0.0,0.0,0.0,1.0,0.0,1.0
5,NV,94.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [91]:
with pd.ExcelWriter('View/Multigig_Sales_View.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    state_summary_view.to_excel(writer, 
                      sheet_name="Sheet1", 
                      startrow=2, 
                      index=False, 
                      header=False) 

In [92]:
# create archived copy
import time

TodaysDate = time.strftime("%m%d%y")

import shutil

excelfilename = "View/archive/" + TodaysDate +"_Multigig_Sales_View.xlsx"
shutil.copyfile('View/Multigig_Sales_View.xlsx', excelfilename)

'View/archive/122122_Multigig_Sales_View.xlsx'